In [1]:
# from google.colab import drive
# drive.mount("/content/drive")
# DRIVE_SRC = "/content/drive/MyDrive/IIT-Projects/third-umpire-decision-automation/backend/experiments/src"
# !rm -r -f ./sample_data
# !cp -r /content/drive/MyDrive/IIT-Projects/third-umpire-decision-automation/.creds/.aws ~/
# !cp -r $DRIVE_SRC ./src

In [2]:
# ! pip install -qq sagemaker

In [3]:
# TODO in the augmented dataset
# 1. Filter out the small bounding boxes
# 2. Split correctly
# 3. There are some images without segmentations

In [4]:
# !rm -r /content/data/Batsmen-Segmentation-Augmented

In [ ]:
import os, shutil
from src.data.download import download
from src.data.generate import segmentationDS2DetectionDS

# download_dir = download("cricket-semantic")
download_dir = "./data/cricket-semantic-segmentation"
segmentationDS2DetectionDS(download_dir)
# seg_dir = f"{download_dir}/segmentations"
# os.makedirs(seg_dir)
# src_img_dir = f"{download_dir}/images"
# src_seg_names = list(filter(lambda nm: "___fuse" in nm, os.listdir(src_img_dir)))
# for src_seg_name in src_seg_names:
#     dst_seg_name = src_seg_name[:-11]
#     src_seg_path = f"{src_img_dir}/{src_seg_name}"
#     dst_seg_path = f"{seg_dir}/{dst_seg_name}"
#     shutil.move(src_seg_path, dst_seg_path)

# imp_fil_names = list(filter(lambda nm: "___" in nm, os.listdir(src_img_dir)))
# for imp_fil_nm in imp_fil_names:
#     imp_fil_path = f"{src_img_dir}/{imp_fil_nm}"
#     os.remove(imp_fil_path)

In [ ]:
raise Exception

In [ ]:
from src.data.generate import makeDarknetSegmentationDataset
from src.data.io import readClassesFile

src_images_root = f"{download_dir}/images"
src_segments_root = f"{download_dir}/segmentations"
src_bboxes_root = f"{download_dir}/bboxes"
classes_path = f"{download_dir}/classes/classes.json"

src_class_names = ["Batsmen", "Ball", "Wicket"]
dst_class_names = ["Batsmen"]
export_path = "./data/Batsmen-Segmentation-Augmented"
classes = readClassesFile(classes_path)

makeDarknetSegmentationDataset(
    src_images_root,
    src_segments_root,
    src_bboxes_root,
    src_class_names,
    dst_class_names,
    classes,
    export_path
)

In [ ]:
!ls /content/data/Batsmen-Segmentation-Augmented/images/

In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt
from src.data.visualize import drawSegments

txt_path = "/content/data/Batsmen-Segmentation-Augmented/segmentations/000000.txt"
img_path = "/content/data/Batsmen-Segmentation-Augmented/images/000000.png"
with open(txt_path) as handler:
  txt = handler.read()
img = cv.imread(img_path)
plt.imshow(drawSegments(img, txt))

In [ ]:
!ls $export_path

In [ ]:
from sagemaker.s3 import S3Uploader
src_dir = export_path
dst_ds_uri = "s3://third-umpire-decision-automation-osura/datasets/batsmen-segmentation"
# S3Uploader.upload(src_dir, dst_ds_uri)

In [ ]:
cropped_img_path = "./data/Batsmen-Segmentation-Augmented[Splitted]/images/train/000000.png"
cropped_lbl_path = "./data/Batsmen-Segmentation-Augmented[Splitted]/segmentations/train/000000.txt"
crp_img = cv.imread(cropped_img_path)
txt_cntnt = ""
with open(cropped_lbl_path) as handler:
    txt_cntnt = handler.read()
drawn_img = drawSegments(crp_img, txt_cntnt)
plt.imshow(drawn_img)
plt.show()